# Getting Data from OECD


In [1]:
import pandas_datareader.data as web
import pandas as pd

C:\Users\nicol\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


### Gross Domestic Expenditure on R&D (GERD) as percentage of GDP

In [2]:
GERD_reader = web.DataReader('MSTI_PUB', 'oecd', start=1981, end=2019)

In [3]:
GERDdf = GERD_reader['GERD as a percentage of GDP']

############# DOUG--
# Compare the initial data here to the rearranged data at the end of this cell.
# This may help with your year formatting?
# Uncomment the csv files to see them
# GERDdf.to_csv('initialGERD.csv')


# Need to change from separate columns for each country to repeated rows
# and a column for the percent value
GERDdf = GERDdf.stack().to_frame()
GERDdf.reset_index(inplace=True)
GERDdf.sort_values(['Country', 'Year'], inplace=True)
GERDdf.reset_index(drop=True, inplace=True)
GERDdf.rename({0:'Percent_GDP_on_RD'}, axis=1, inplace=True)

GERDdf.head()

# GERDdf.to_csv('formattedGERD.csv')

,Year,Country,Percent_GDP_on_RD
0,1989-01-01,Argentina,0.385030
1,1990-01-01,Argentina,0.379223
2,1991-01-01,Argentina,0.404291
3,1992-01-01,Argentina,0.359716
4,1993-01-01,Argentina,0.403760


### Percent of Population with Health Insurance (Public or Private)

In [4]:
HI_reader = web.DataReader('HEALTH_PROT', 'oecd', start=1981, end=2019)

In [5]:
HIdf = HI_reader['Total public and primary private health insurance']['% of total population covered']

# Need to change from separate columns for each country to repeated rows
# and a column for the percent value
HIdf = HIdf.stack().to_frame()
HIdf.reset_index(inplace=True)
HIdf.sort_values(['Country', 'Year'], inplace=True)
HIdf.reset_index(drop=True, inplace=True)
HIdf.rename({0:'Percent_Pop_HI'}, axis=1, inplace=True)

HIdf.head()

,Year,Country,Percent_Pop_HI
0,1981-01-01,Australia,100.0
1,1982-01-01,Australia,100.0
2,1983-01-01,Australia,100.0
3,1984-01-01,Australia,100.0
4,1985-01-01,Australia,100.0


### Production-based CO2 productivity, GDP per unit of energy-related CO2 emissions

In [6]:
CO2_reader = web.DataReader('GREEN_GROWTH', 'oecd', start=1981, end=2019)

In [7]:
# This one is different because each country had a separate table
# instead of all being a column in one table. Below is the code that
# combines all of the tables into one dataframe.


# Initializing empty dataframe
CO2df = pd.DataFrame(columns=['Year', 'Country', 'GDP_per_unit_CO2'])

# CO2_reader has a multi-index. The country is the first entry of each,
# so the following code pulls out the country names and set() gets rid of duplicates.
countries = set(CO2_reader.columns.get_level_values(0))
countries = sorted(list(countries))

# This for loop gets the particular data we are interested in for 
# each country and then adds that to the initialized/master dataframe:
for country in countries:
    Cseries = CO2_reader[country]['Production-based CO2 productivity, GDP per unit of energy-related CO2 emissions']
    Cdf = Cseries.to_frame()
    Cdf.reset_index(inplace=True)
    Cdf.rename({'Production-based CO2 productivity, GDP per unit of energy-related CO2 emissions':'GDP_per_unit_CO2'},
                 axis=1, inplace=True) 
    Cdf.insert(1, 'Country', country)
    CO2df = pd.concat([CO2df, Cdf], axis=0)

CO2df.head()

,Year,Country,GDP_per_unit_CO2
0,1990-01-01,ASEAN,NaN
1,1991-01-01,ASEAN,NaN
2,1992-01-01,ASEAN,NaN
3,1993-01-01,ASEAN,NaN
4,1994-01-01,ASEAN,NaN


### Merging Data Together

In [8]:
merge1 = pd.merge(left=GERDdf, right=HIdf, how='outer',
                       left_on=['Country', 'Year'], right_on=['Country', 'Year'])
OECDdf = pd.merge(left=merge1, right=CO2df, how='outer',
                       left_on=['Country', 'Year'], right_on=['Country', 'Year'])
OECDdf.sort_values(['Country', 'Year'], inplace=True)
OECDdf.reset_index(drop=True, inplace=True)

OECDdf.head()

,Year,Country,Percent_GDP_on_RD,Percent_Pop_HI,GDP_per_unit_CO2
0,1990-01-01,ASEAN,NaN,NaN,NaN
1,1991-01-01,ASEAN,NaN,NaN,NaN
2,1992-01-01,ASEAN,NaN,NaN,NaN
3,1993-01-01,ASEAN,NaN,NaN,NaN
4,1994-01-01,ASEAN,NaN,NaN,NaN


### Fixing Country Names

In [9]:
import pycountry
# Official/Up-to-date Country Names
pycountries = [country.name for country in pycountry.countries]

# Country names that appear in OECD data
OECDcountries = sorted(list(set(OECDdf.iloc[:,1])))

# Making csv for me to see what needs to be changed
Same = sorted(list(set(OECDcountries).intersection(set(pycountries))))
OECD_Only = sorted(list(set(OECDcountries) - set(pycountries)))
Py_Only = sorted(list(set(pycountries) - set(OECDcountries)))
comparisondf = pd.DataFrame({'Same': pd.Series(Same),
                             'Merged_Only': pd.Series(OECD_Only),
                             'Py_Only': pd.Series(Py_Only)})
# comparisondf.to_csv('comp.csv')

In [10]:
# OECD has some groups of countries that I will remove
remove = ['ASEAN', 'BRIICS economies - Brazil, Russia, India, Indonesia, China and South Africa',
         'Eastern Europe, Caucasus and Central Asia', 'Euro area (19 countries)',
         'European Union (15 countries)', 'European Union (28 countries)',
         'G20', 'G7', 'Latin America and Caribbean', 'Middle East and North Africa',
         'OECD - Europe', 'OECD - Total', 'OECD America', 'OECD Asia Oceania', 'World']

OECDdf = OECDdf[~OECDdf.Country.isin(remove)]

In [11]:
# Other country names need to be updated to match current official names
update_dic = {'Bolivia':'Bolivia, Plurinational State of',
              "China (People's Republic of)":'China',
              'Chinese Taipei':'Taiwan, Province of China',
              'Czech Republic':'Czechia',
              'Democratic Republic of the Congo':'Congo, The Democratic Republic of the',
              'Iran':'Iran, Islamic Republic of',
              'Korea':'Korea, Republic of',
              'Micronesia':'Micronesia, Federated States of',
              'Moldova':'Moldova, Republic of',
              'Russia':'Russian Federation',
              'Slovak Republic':'Slovakia',
              'Tanzania':'Tanzania, United Republic of',
              'Venezuela':'Venezuela, Bolivarian Republic of'}
OECDdf = OECDdf.replace({"Country": update_dic})
OECDdf.sort_values(['Country', 'Year'], inplace=True)
OECDdf.reset_index(drop=True, inplace=True)

In [12]:
# Checking that it worked
OECDcountries = sorted(list(set(OECDdf.iloc[:,1])))
Same = sorted(list(set(OECDcountries).intersection(set(pycountries))))
OECD_Only = sorted(list(set(OECDcountries) - set(pycountries)))
Py_Only = sorted(list(set(pycountries) - set(OECDcountries)))
comparisondf2 = pd.DataFrame({'Same': pd.Series(Same),
                             'Merged_Only': pd.Series(OECD_Only, dtype='float64'),
                             'Py_Only': pd.Series(Py_Only)})
# comparisondf2.to_csv('comp2.csv')
# All good, except that there are pycountries not in the OECD data

In [13]:
# Now I want to add all countries in pycountries (and all years from 1982),
# but I'm going to convert the Year column from timestamp to integer first.

OECDdf['Year'] = OECDdf['Year'].dt.year

In [14]:
# Now I'll make a dataframe of all the pycountries repeated 
# for each year from 1981-2019 that can then be joined with OECDdf

# Makes a dataframe with each year repeated as many
# times as there are countries (249 times)
years = pd.Series(range(1981,2020), name='Year')
years2 = years.repeat(len(pycountries))
years_countries_df = years2.to_frame()

# Cycles through country names to add one for each year
from itertools import cycle
seq = cycle(pycountries)
years_countries_df['Country'] = [next(seq) for count in range(years_countries_df.shape[0])]

# Looking at resulting dataframe
years_countries_df.reset_index(drop=True, inplace=True)
years_countries_df.head()

,Year,Country
0,1981,Aruba
1,1981,Afghanistan
2,1981,Angola
3,1981,Anguilla
4,1981,Åland Islands


In [15]:
# Merging with OECD data so that all countries and years appear,
# even if the variables are missing
OECDdf = pd.merge(left=OECDdf, right=years_countries_df, how='outer',
                       left_on=['Country', 'Year'], right_on=['Country', 'Year'])
OECDdf.sort_values(['Country', 'Year'], inplace=True)
OECDdf.reset_index(drop=True, inplace=True)

### Dealing with Missing Data

In [16]:
OECDdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9711 entries, 0 to 9710
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year               9711 non-null   int64  
 1   Country            9711 non-null   object 
 2   Percent_GDP_on_RD  1248 non-null   float64
 3   Percent_Pop_HI     1099 non-null   float64
 4   GDP_per_unit_CO2   3709 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 379.5+ KB


In [17]:
# I'm going to just drop the R&D and Health Insurance columns
# since they are missing so many years/countries
OECDdf.drop(columns=['Percent_GDP_on_RD', 'Percent_Pop_HI'], inplace=True)

In [18]:
# I'm also going to drop 2019, since Vid did in the World Bank data

# Get names of indexes for which column Year has value 2019
indexNames = OECDdf[ OECDdf['Year'] == 2019 ].index
# Delete these row indexes from df
OECDdf.drop(indexNames, inplace=True)

In [20]:
# I will keep the GDP per unit of CO2 column, since it 
# has a significant number of countries (usually from 1990 on).

# I will get the countries that actually have this data into
# a list for reference.
non_null = OECDdf.dropna()
nn_countries = sorted(list(set(non_null['Country'])))
len(nn_countries) # 133 countries have this data for at least some years

# Generally, we have this data from 1990-2018, but some countries are
# missing some of those years.
nn_years = sorted(list(set(non_null['Year'])))

In [21]:
# Most countries that were missing years were missing 1990 and/or 2018.
# I'm going to remove countries missing more than those two years.
# For the countries missing 1990, I will fill it with their value from 1991. 
# For the countries missing 2018, I will fill it with their value from 2017.

# This for loop does the above three tasks.

for country in nn_countries:
    
    # list of available years for a particular country
    av_years = list(non_null.loc[non_null['Country'] == country, 'Year'])
    
    # missing years for the particular country
    mi_years = list(set(nn_years) - set(av_years))
    
    # Removing countries missing more than 1990 and 2018:
    if len(mi_years) > 2:
        indexNames = non_null[ non_null['Country'] == country ].index
        non_null.drop(indexNames, inplace=True)
    
    # Filling in missing 1990 values:
    elif 1990 in mi_years:
        val91 = non_null.loc[(non_null['Country']==country) & (non_null['Year']==1991), 'GDP_per_unit_CO2'].values[0]
        non_null = non_null.append({'Year':1990, 'Country':country, 'GDP_per_unit_CO2':val91}, ignore_index=True)
    
    # Filling in missing 2018 values:
    elif 2018 in mi_years:
        val17 = non_null.loc[(non_null['Country']==country) & (non_null['Year']==2017), 'GDP_per_unit_CO2'].values[0]
        non_null = non_null.append({'Year':2018, 'Country':country, 'GDP_per_unit_CO2':val17}, ignore_index=True)
        
non_null.sort_values(['Country', 'Year'], inplace=True)
non_null.reset_index(drop=True, inplace=True)

### Writing to CSV

In [23]:
non_null.to_csv('OECD_Clean.csv', index=False)